In [2]:
%cd /home/sagemaker-user/MMDetection
import boto3           # For interacting with S3
import pandas as pd
import sys             # Python system library needed to load custom functions
sys.path.append('..')  # Add the source directory to the PYTHONPATH. This allows to import local functions and modules.
import sagemaker
# Imports to run Sagemaker training jobs
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
from sagemaker.session import Session, TrainingInput
import tarfile
from Dataset import OnchoDataset
from detection_util import create_predictions
from gdsc_score import get_leaderboard_score
from gdsc_util import download_directory, download_file, load_sections_df, set_up_logging, PROJECT_DIR
from PredictionEvaluator import PredictionEvaluator
from config import DEFAULT_BUCKET, DEFAULT_REGION  # The name of the S3 bucket that contains the training data
from detection_util import create_predictions
from gdsc_util import download_and_extract_model, set_up_logging, extract_hyperparams,create_encrypted_bucket, PROJECT_DIR
from training_4k import load_config as load_config_4k
set_up_logging()  # Sets up logging to console and the .log file
import matplotlib.pyplot as plt  # Used for plotting
import mmcv  # Object detection framework
import os  # Interaction with the file system
import pandas as pd  # Home of the DataFrame construct, _the_ most important object for Data Science
import sys  # Python system library needed to load custom functions

from matplotlib.patches import Rectangle  # Allows drawing the bounding boxes of the worm sections
from mmcv import Config  # Loading and accessing MMDetection configuration files
from mmdet.apis import inference_detector, init_detector, train_detector, set_random_seed  # Part of the MMDetection framework
from mmdet.datasets import build_dataset  # Part of the MMDetection framework
from mmdet.models import build_detector  # Part of the MMDetection framework

from PIL import Image  # For loading image files
from tqdm import tqdm  # for timing a for loop

/home/sagemaker-user/MMDetection
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [3]:
!pip install imutils
!pip install docutils
!pip install -U sagemaker

In [4]:
%pwd

'/home/sagemaker-user/MMDetection'

In [5]:
%%time

data_folder = str(PROJECT_DIR / 'MMDetection/data')
cfg, base_file = load_config_4k(data_folder)

print(f'data folder: {data_folder}')
print(f'max boxes per imgage: {cfg.model.test_cfg.rcnn.max_per_img}')
print(f'epochs: {cfg.runner.max_epochs}')
print(f'batch size: {cfg.data.samples_per_gpu}')
print(f"image size: {cfg.train_pipeline[3]['img_scale']}")

data folder: /home/sagemaker-user/MMDetection/data
max boxes per imgage: 400
epochs: 5
batch size: 1
image size: (4096, 4096)
CPU times: user 2 s, sys: 946 ms, total: 2.94 s
Wall time: 20.2 s


In [6]:
%cd /home/sagemaker-user/MMDetection/data

/home/sagemaker-user/MMDetection/data


In [7]:
entry_point = 'training_4k.py'
exp_name = entry_point.split('.')[0].replace('_', '-')  # AWS does not allow . and _ as experiment names
exp_name

'training-4k'

In [8]:
sagemaker_session = sagemaker.Session()
sagemaker_role = sagemaker.get_execution_role()
bucket = 'sagemaker-studio-jna7gyyxsih'
print(f'SageMaker Session: {sagemaker_session}')
print(f'Role: {sagemaker_role}')
print(f'Bucket: {bucket}')

s3_input = "s3://sagemaker-studio-jna7gyyxsih/mmdetection/mmd-input/training-input".format(bucket)
s3_images = "s3://sagemaker-studio-jna7gyyxsih/mmdetection/mmd-input/dataset" # Images files are here, in a subfolder named 'train'
s3_output = 's3://sagemaker-studio-jna7gyyxsih/mmdetection/mmd-output'.format(bucket)

cfg='{}/input/models/'.format(s3_input)
images='{}/'.format(s3_images)
outpath='{}/'.format(s3_output)

print(cfg)
print(images)
print(outpath)

SageMaker Session: <sagemaker.session.Session object at 0x7f8629d5b828>
Role: arn:aws:iam::494549386743:role/service-role/AmazonSageMaker-ExecutionRole-20220618T161542
Bucket: sagemaker-studio-jna7gyyxsih
s3://sagemaker-studio-jna7gyyxsih/mmdetection/mmd-input/training-input/input/models/
s3://sagemaker-studio-jna7gyyxsih/mmdetection/mmd-input/dataset/
s3://sagemaker-studio-jna7gyyxsih/mmdetection/mmd-output/


In [9]:
def make_tarfile_without_parent(output_filename, source_dir, excluded_paths):
    with tarfile.open(output_filename, "w:gz") as tar:
        for fn in os.listdir(source_dir):
            p = os.path.join(source_dir, fn)
            if p not in excluded_paths:
                tar.add(p, arcname=fn)

In [10]:
%cd /home/sagemaker-user

# the folders 'data' and 'output' are big and not supposed to be there

excluded_paths=['/home/sagemaker-user/MMDetection/data',
               '/home/sagemaker-user/MMDetection/output']

make_tarfile_without_parent("input.tar.gz", "/home/sagemaker-user/MMDetection/",excluded_paths)

/home/sagemaker-user


In [11]:
import boto3, os
s3 = boto3.resource('s3')
s3.meta.client.upload_file("input.tar.gz","sagemaker-studio-jna7gyyxsih", "mmdetection/mmd-input/training-input/input/input.tar.gz")

In [12]:
inputs = {"images": TrainingInput(images)}

## Change Epochs by: runner.max_epochs in the next cell

In [13]:
entry_point = 'training_4k.py'
exp_name = entry_point.split('.')[0].replace('_', '-')  # AWS does not allow . and _ as experiment names

# account_id = boto3.client('sts').get_caller_identity().get('Account')
role = get_execution_role()

# sm_client = boto3.client("sagemaker", region_name=DEFAULT_REGION)
# sess = Session(sagemaker_client=sm_client)
# s3_output_location = f"s3://{sess.default_bucket()}/{exp_name}"
# input_channels = {"train": f"s3://{DEFAULT_BUCKET}"}

#hyperparameters = extract_hyperparams(entry_point) # custom function to parse the training script and extract config
hyperparameters = {
    'dataset_type': "OnchoDataset",
    'data_root': 'dataset/',
    'data.train.type': "OnchoDataset",
    'data.train.data_root': 'dataset/',
    'data.train.img_prefix': "train/",
    'data.train.ann_file': "gdsc_train.csv",
    'data.test.type': "OnchoDataset",
    'data.test.data_root': 'dataset/',
    'data.test.img_prefix': "train/",
    'data.test.ann_file': "gdsc_train.csv",
    'data.val.type': "OnchoDataset",
    'data.val.data_root': 'dataset/',
    'data.val.img_prefix': "train/",
    'data.val.ann_file': "gdsc_train.csv",
    'load_from': 'mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth',
    'work_dir': '/opt/ml/model/',
    'seed': '0',
    'data.samples_per_gpu': '1',
    'data.workers_per_gpu': '1',
    'model.roi_head.bbox_head.num_classes': '1',
    'lr_config.warmup': 'None',
    'log_config.interval': '50',
    'evaluation.metric': "'mAP'",
    'evaluation.interval': '1',
    'checkpoint_config.interval': '1',
    'runner.max_epochs': '1',
    'device': 'cuda',
    'model.test_cfg.rcnn.max_per_img': '400'
}
hyperparameters['base_file'] = base_file

metrics = [
    {"Name": "train:loss_rpn_cls", "Regex": "loss_rpn_cls: ([0-9\.]+)"},
    {"Name": "train:loss_rpn_bbox", "Regex": "loss_rpn_bbox: ([0-9\.]+)"},
    {"Name": "train:loss_cls", "Regex": "loss_cls: ([0-9\.]+)"},
    {"Name": "train:loss_bbox", "Regex": "loss_bbox: ([0-9\.]+)"},
    {"Name": "train:loss", "Regex": "loss: ([0-9\.]+)"},
    {"Name": "train:accuracy", "Regex": "acc: ([0-9\.]+)"},
    {"Name": "train:epoch", "Regex": "Epoch (\[[0-9\.]+\])"},
    {"Name": "val:epoch", "Regex": "Epoch\(val\) (\[[0-9]+\])"},
    {"Name": "val:mAP", "Regex": "mAP: ([0-9\.]+)"},
]

estimator = PyTorch(
    entry_point=entry_point,             # This function will be called by the training job
    source_dir='s3://sagemaker-studio-jna7gyyxsih/mmdetection/mmd-input/training-input/input/input.tar.gz',
    image_uri=f"954362353459.dkr.ecr.{DEFAULT_REGION}.amazonaws.com/sm-training-custom:torch-1.8.1-cu111-noGPL",
    input_mode='File',
    role=role,
    output_path=outpath,
    container_log_level=20,             # 10=debug, 20=info
    base_job_name=exp_name,
    instance_count=1,
    instance_type="ml.g4dn.xlarge",     # a GPU instance
    volume_size=45,
    metric_definitions=metrics,
    hyperparameters=hyperparameters,
)

In [14]:
estimator.fit(
    inputs,
    wait=False,           # Whether or not the notebook should wait for the job to finish. By setting it to False we can continue working while the job runs on another machine.
)

job_name=f'{estimator._hyperparameters["sagemaker_job_name"]}'
model_location = f'{outpath}{job_name}/output/model.tar.gz'
print(model_location)

path='/home/sagemaker-user/MMDetection/output/'+job_name
os.mkdir(path)

with open(f'{PROJECT_DIR}/MMDetection/output/{job_name}/model_location.txt', 'w+') as f:
    f.write(model_location)

2022-08-14 15:00:57,528 - sagemaker.image_uris - INFO - Defaulting to the only supported framework/algorithm version: latest.
2022-08-14 15:00:57,570 - sagemaker.image_uris - INFO - Ignoring unnecessary instance type: None.
2022-08-14 15:00:57,579 - sagemaker - INFO - Creating training-job with name: training-4k-2022-08-14-15-00-57-527
s3://sagemaker-studio-jna7gyyxsih/mmdetection/mmd-output/training-4k-2022-08-14-15-00-57-527/output/model.tar.gz


# If the notebook stopped, and training is done, just modify the Job_name in the next cell and switch to GPU for detection

In [18]:
job_name='training-4k-2022-08-14-15-00-57-527'  # fix it manually 

model_location=f's3://sagemaker-studio-jna7gyyxsih/mmdetection/mmd-output/{job_name}/output/model.tar.gz'
local_directory=f"MMDetection/output/{job_name}"
local_model_dir = download_and_extract_model(model_uri=model_location, local_dir=local_directory)
local_model_dir

2022-08-12 18:42:10,464 - gdsc_util - INFO - File mmdetection/mmd-output/training-4k-2022-08-10-15-08-48-262/output/model.tar.gz already exists. Skipping download


'/home/sagemaker-user/MMDetection/output/training-4k-2022-08-10-15-08-48-262/mmd-output'

In [5]:
# def getAllFiles(path,extension='jpg'):
#     import glob, os
#     os.chdir(path)
#     return [file for file in glob.glob('*.'+extension)]

In [19]:
%pwd

'/home/sagemaker-user'

In [7]:
# root_path='/home/sagemaker-user/'
# job_name='training-4k-2022-08-07-17-17-20-543'
# source_train_folder=root_path+'river-blindness/content/new/dataset/train/'
# source_valid_folder=root_path+'river-blindness/content/new/dataset/valid/'
# source_test_folder=root_path+'river-blindness/yolov5/test/images/'

# train_imgs=getAllFiles(source_train_folder,extension='jpg')
# valid_imgs=getAllFiles(source_valid_folder,extension='jpg')
# test_imgs=getAllFiles(source_test_folder,extension='jpg')

# train_valid_images=train_imgs+valid_imgs # for fixing the labels

In [23]:
# to  create labels, we need to run the detection on the train (not test) images
# train_path_csv='/home/sagemaker-user/MMDetection/data/train_files.csv'
# pd.DataFrame(train_valid_images).to_csv(train_path_csv,sep=';', header=None,index=False)

In [20]:
%%time

# get the final/last epochs file
data_folder = str(PROJECT_DIR / 'MMDetection/data')
config, base_file = load_config_4k(data_folder)
print(f'Number of epochs: {config.runner.max_epochs}')
last_epoch=f"epoch_{config.runner.max_epochs}.pth"
last_epoch='epoch_8.pth'

Number of epochs: 20
CPU times: user 1.29 s, sys: 748 ms, total: 2.04 s
Wall time: 19.3 s


# Check the output of previous cell whether we're using the last epoch

In [21]:
%cd 

/root


In [22]:
job_name='training-4k-2022-08-10-15-08-48-262'
local_model_dir=f"/home/sagemaker-user/MMDetection/output/{job_name}/mmd-output"
checkpoint = f'{local_model_dir}/{last_epoch}' # Select one of the model checkpoints to load in
file_names = pd.read_csv(f'{data_folder}/test_files.csv', sep=';', header=None)[0].values

# To run labels uncomment and run the following cell

In [ ]:
# %%time 

# local_model_dir='/home/sagemaker-user/MMDetection/output/training-4k-2022-08-07-17-17-20-543/mmd-output'
# checkpoint = f'{local_model_dir}/{last_epoch}'
# file_names_train=pd.read_csv(f'{data_folder}/train_files.csv', sep=';', header=None)[0].values
# prediction_df_train = create_predictions(file_names_train, config, checkpoint)
# %cd /home/sagemaker-user/
# local_directory=f"MMDetection/output/{job_name}"
# file_name='updated-labels-'+ job_name + '.csv'
# prediction_df_train.to_csv(f'{local_directory}/{file_name}', sep=';')

load checkpoint from local path: /home/sagemaker-user/MMDetection/output/training-4k-2022-08-07-17-17-20-543/mmd-output/epoch_8.pth
2022-08-08 20:43:18,862 - detection_util - INFO - Creating predictions for 994 files


  0%|          | 0/994 [00:00<?, ?it/s]

2022-08-08 20:43:18,866 - detection_util - INFO - Processing file: 79_B.jpg


/mmdetection/mmdet/datasets/utils.py:70: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)


[2022-08-08 20:43:20.957 gdsc5-smstudio-cust-ml-g4dn-xlarge-f78d9f621f45e8ce75aa4230fd29:61 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-08-08 20:43:21.016 gdsc5-smstudio-cust-ml-g4dn-xlarge-f78d9f621f45e8ce75aa4230fd29:61 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


...

2022-08-09 08:04:49,674 - detection_util - INFO - Processing file: 52_B.jpg


 86%|████████▌ | 850/994 [11:22:21<2:06:26, 52.69s/it]

2022-08-09 08:05:40,572 - detection_util - INFO - Processing file: 221_D.jpg


 86%|████████▌ | 851/994 [11:23:08<2:01:07, 50.82s/it]

2022-08-09 08:06:27,039 - detection_util - INFO - Processing file: 35_A.jpg


In [23]:
%pwd

'/root'

# We need to add higth and width to the new csv file

In [64]:
# %cd /home/sagemaker-user
# job_name='training-4k-2022-08-07-17-17-20-543'
# local_model_dir=f'MMDetection/output/{job_name}'
# file_name='updated-labels-'+ job_name + '.csv'
# prediction_df_train=pd.read_csv(f'{local_model_dir}/{file_name}', sep=';')
# prediction_df_train["height"] = 0
# prediction_df_train["width"] = 0
# section_df=pd.read_csv('MMDetection/gdsc_train.csv', sep=';')
# all_imgs=list(section_df['file_name'].unique())
# dict_h_w={}
# for index,row in section_df.iterrows():
#     dict_h_w[row['file_name']]=(row['height'],row['width'])
    
# print(f"Size of dict: {len(dict_h_w)}")

# for index,row in prediction_df_train.iterrows():
#     prediction_df_train.at[index,'height']=dict_h_w[row['file_name']][0]
#     prediction_df_train.at[index,'width']=dict_h_w[row['file_name']][1]

# prediction_df_train
# prediction_df_train.to_csv(f'{local_model_dir}/{file_name}', sep=';')

# Running the detection

In [24]:
%%time 
prediction_df = create_predictions(file_names, config, checkpoint,device='cpu')

load checkpoint from local path: /home/sagemaker-user/MMDetection/output/training-4k-2022-08-10-15-08-48-262/mmd-output/epoch_8.pth
2022-08-12 18:43:59,935 - detection_util - INFO - Creating predictions for 73 files


  0%|          | 0/73 [00:00<?, ?it/s]

2022-08-12 18:43:59,939 - detection_util - INFO - Processing file: 100_D.jpg


  1%|▏         | 1/73 [00:30<36:17, 30.25s/it]

2022-08-12 18:44:30,187 - detection_util - INFO - Processing file: 100_C.jpg


...

2022-08-12 19:33:02,460 - detection_util - INFO - Processing file: 99_AA.jpg


 99%|█████████▊| 72/73 [49:30<00:33, 33.16s/it]

2022-08-12 19:33:30,030 - detection_util - INFO - Processing file: 99_A.jpg


100%|██████████| 73/73 [50:15<00:00, 41.31s/it]

CPU times: user 55min 41s, sys: 18min 27s, total: 1h 14min 9s
Wall time: 50min 18s


In [10]:
prediction_df

,file_name,xmin,ymin,xmax,ymax,detection_score
section_id,,,,,,
100_D.jpg@3047-3269-3616-3789,100_D.jpg,3047,3616,3269,3789,0.998979
100_D.jpg@1629-1806-3187-3458,100_D.jpg,1629,3187,1806,3458,0.998705
100_D.jpg@2896-3086-3137-3327,100_D.jpg,2896,3137,3086,3327,0.998632
100_D.jpg@3046-3278-4087-4313,100_D.jpg,3046,4087,3278,4313,0.998483
100_D.jpg@2677-2839-6258-6460,100_D.jpg,2677,6258,2839,6460,0.998259
...,...,...,...,...,...,...
99_A.jpg@2046-2122-6229-6318,99_A.jpg,2046,6229,2122,6318,0.053920
99_A.jpg@4309-4662-2018-2871,99_A.jpg,4309,2018,4662,2871,0.053610
99_A.jpg@4187-4255-1060-1152,99_A.jpg,4187,1060,4255,1152,0.052414


In [25]:
%cd /home/sagemaker-user/
local_directory=f"MMDetection/output/{job_name}"
file_name='submission-'+ job_name + '.csv'

prediction_df.drop_duplicates(subset=['file_name', 'xmin', 'xmax', 'ymin', 'ymax'], inplace=True)
prediction_df.to_csv(f'{local_directory}/{file_name}', sep=';')

/home/sagemaker-user


In [16]:
# !conda install -y -c conda-forge zip

In [15]:
# !zip -r -X /home/sagemaker-user/MMDetection/dataall_processed-rgb_images.zip /home/sagemaker-user/MMDetection/data/processed-rgb-images

In [14]:
# !split --help

In [13]:
# !zip -U -s 200M -O /home/sagemaker-user/MMDetection/data/split-zip/processed_rgb_images_part_.zip /home/sagemaker-user/MMDetection/dataall_processed-rgb_images.zip